In [97]:
import numpy as np
import pandas as pd
#构建数据集
#用2表示Sunny  和  Hot
#用数字1表示 Mild 和 High 和 strong,overcast
#用数字0表示  Rian 和 cool和 Nolmal
data=pd.DataFrame([
                  [2,2,1,0,0],
                  [2,2,1,1,0],
                  [1,1,1,0,1],
                  [0,1,1,0,1],
                  [0,0,0,0,1],
                  [0,0,0,1,0],
                  [1,0,0,1,1],
                  [2,1,1,0,0],
                  [2,0,0,0,1],
                  [0,1,0,0,1],
                  [2,1,0,1,1],
                  [1,1,1,1,1],
                  [1,2,0,0,1],
                  [0,1,1,1,0]
                  ])
name=['Outlook','Temperature','Humidity','Wind','Playtennis']
data.columns=name

In [98]:
X = data.drop(columns=['Playtennis'])
Y=data['Playtennis']
featurenames=X.columns
X_test = pd.DataFrame(data=[[2, 0, 1, 1]], columns=['Outlook', 'Temperature', 'Humidity', 'Wind'])

In [99]:
X

,Outlook,Temperature,Humidity,Wind
0,2,2,1,0
1,2,2,1,1
2,1,1,1,0
3,0,1,1,0
4,0,0,0,0
5,0,0,0,1
6,1,0,0,1
7,2,1,1,0
8,2,0,0,0
9,0,1,0,0


In [100]:
Y

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Playtennis, dtype: int64

In [101]:
X_test

,Outlook,Temperature,Humidity,Wind
0,2,0,1,1


In [102]:
#计算先验
def NB(X,Y):
    y_unique=np.unique(Y)
    prior=np.zeros(len(y_unique))
#计算先验信息
    for i in range(len(y_unique)):
        prior[i]=sum(Y==y_unique[i])/len(Y)
#计算似然
    condition={}
    for i in featurenames:
        X_unique=list(set(X[i]))
        X_condition= np.zeros((len(y_unique),len(X_unique)))
        for j in range(len(y_unique)):
            for k in range(len(X_unique)):
                X_condition[j, k] = sum((X[i] == X_unique[k]) & (Y == y_unique[j])) / sum(Y == y_unique[j])
        X_condition=pd.DataFrame(X_condition,columns=X_unique,index=y_unique)
        condition[i]=X_condition
    return prior,condition

In [103]:
def test_Model(X_test, prior, condition):
    featurenames = X_test.columns
    post_prob = np.zeros((X_test.shape[0], prior.shape[0]))
    for k in range(X_test.shape[0]):
        for i in range(prior.shape[0]):
            p = prior[i]
            for j in range(len(featurenames)):
                value = X_test.iloc[k, j]
                cp = condition[featurenames[j]]
                cp_val = cp.iloc[i, value]
                p *= cp_val
            post_prob[k, i] = p
        # 归一化操作
        post_prob[k, :] /= np.sum(post_prob[k, :])
    return post_prob

In [104]:
prior,condition=NB(X,Y)
P=test_Model(X_test,prior,condition)
print(f'先验概率为{prior}\n似然为：{condition}\n预测结果为：{P}')

先验概率为[0.35714286 0.64285714]
似然为：{'Outlook':           0         1         2
0  0.400000  0.000000  0.600000
1  0.333333  0.444444  0.222222, 'Temperature':           0         1         2
0  0.200000  0.400000  0.400000
1  0.333333  0.555556  0.111111, 'Humidity':           0         1
0  0.200000  0.800000
1  0.666667  0.333333, 'Wind':           0         1
0  0.400000  0.600000
1  0.666667  0.333333}
预测结果为：[[0.79541735 0.20458265]]
